# Download dos dados de setores sensitários do IBGE

Os dados de setores sensitários são baixados diretamente do ftp do IBGE.

In [1]:
# Importando Bibliotecas necessárias
from os import path, sep, makedirs
from ftplib import FTP
import re
from tqdm import tqdm
import pandas as pd
import geopandas as gpd


In [2]:
ano = 2010
base_path = "./SETORES_CENSITARIOS"

ftp = {
    2010: {
        "ftp_host": "geoftp.ibge.gov.br",
        "directory": "organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2010/setores_censitarios_shp",
    },
    2022: {
        "ftp_host": "ftp.ibge.gov.br",
        "directory": "/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/setores/gpkg/UF",
    },
}

states = [
    "AC",
    "AL",
    "AM",
    "AP",
    "BA",
    "CE",
    "DF",
    "ES",
    "GO",
    "MA",
    "MG",
    "MS",
    "MT",
    "PA",
    "PB",
    "PE",
    "PI",
    "PR",
    "RJ",
    "RN",
    "RO",
    "RR",
    "RS",
    "SC",
    "SE",
    "SP",
    "TO",
    "SP",
]

# Baixando os dados de setores censitários

Agora vamos baixar os arquivos georreferenciados dos setores sensitários, para utilizar menos memória RAM durante o processo de geração de pontos, vamos baixar cada estado separadamente.

In [3]:
pattern = re.compile(
    r"^[A-Z]{2}$", re.IGNORECASE
)  # padrao para filtrar arquivos que iniciam com a sigla do estado
folder = path.join(base_path, str(ano))  # pasta onde os arquivos serao salvos
if ano == 2022:
    folder = path.join(folder, "gpkg")

# Criar pasta para salvar os arquivos caso não exista
makedirs(folder, exist_ok=True)


def download_ftp_files(ftp_host, directory="/", dest_folder=folder, chunk_size=1024):
    """Lista arquivos em um diretório FTP e baixa os arquivos .zip retornando uma lista de arquivos baixados

    ARGS:
        ftp_host (str): Endereço do servidor FTP
        directory (str, optional): Diretório a ser listado. Defaults to "/".
        dest_folder (str, optional): Pasta onde os arquivos serão salvos. Defaults to folder.
        chunk_size (int, optional): Tamanho do buffer para download. Defaults to 1024.
    RETURNS:
        list: Lista de arquivos baixados
        None: Se ocorrer um erro

    """
    try:
        # Conectar ao servidor FTP
        ftp = FTP(ftp_host)
        ftp.login()  # Login anônimo

        # Navegar para o diretório desejado
        ftp.cwd(directory)

        items = []
        # listando todas as pastas no diretório
        ftp.retrlines("LIST", items.append)
        files = []
        count = 0
        for item in tqdm(items, desc="Baixando arquivos", unit="arquivos", leave=True):
            count += 1
            parts = item.split()
            item_name = parts[-1]
            # Verifica se o item é um diretório e se o nome for a sigla do estado
            if item.startswith("d") and re.match(pattern, item_name):
                # entra no diretório do estado
                ftp.cwd(item_name)
                # listar arquivos no diretório
                state_files = ftp.nlst()
                # Filtrar arquivos que contenham "setores" no nome
                state_files = [f for f in state_files if "setores" in f]
                # Baixa os arquivos localmente
                for file in state_files:
                    tqdm.write(file)
                    local_path = path.join(dest_folder, file)
                    with open(local_path, "wb") as f:
                        ftp.retrbinary(
                            f"RETR {file}",
                            lambda data: f.write(data),
                            blocksize=chunk_size,
                        )
                    files.append(local_path)
                # sair do diretório do estado para diretório principal
                ftp.cwd("..")
        ftp.quit()
        print(f" {len(files)} arquivos baixados com sucesso!")
        return files
    except Exception as e:
        print(f"Erro: {e}")
        return None


# Endereço do servidor FTP e diretório
ftp_host = ftp[ano]["ftp_host"]
directory = ftp[ano]["directory"]

# Baixa os arquivos do FTP
files_to_download = download_ftp_files(ftp_host, directory)

Baixando arquivos:   0%|          | 0/28 [00:00<?, ?arquivos/s]

ac_setores_censitarios.zip


Baixando arquivos:   7%|▋         | 2/28 [00:00<00:06,  3.76arquivos/s]

al_setores_censitarios.zip


Baixando arquivos:  11%|█         | 3/28 [00:01<00:13,  1.88arquivos/s]

...

sp_setores_censitarios.zip


Baixando arquivos:  96%|█████████▋| 27/28 [00:42<00:03,  3.83s/arquivos]

to_setores_censitarios.zip


Baixando arquivos: 100%|██████████| 28/28 [00:42<00:00,  1.53s/arquivos]

 27 arquivos baixados com sucesso!


# Gerando o arquivo geopackage com os setores sensitários

Caso o ano seja 2010, o arquivo geopackage é gerado a partir dos arquivos shapefiles baixados. 
Caso o ano seja 2020, o arquivo geopackage é baixado diretamente do ftp do IBGE e não necessita conversão e nem descompactação do mesmo.

In [5]:
# Gera geopacakge com os arquivos baixados
if ano == 2010:
    for file in tqdm(
        files_to_download, desc="Descompactando arquivos", unit="arquivo", leave=True
    ):
        zip_file = f"zip://{file}"
        filename = file.split(sep)[-1]
        tqdm.write(filename)
        otp_file = filename.replace(".zip", ".gpkg").replace(" ", "")
        makedirs(path.join(folder, "gpkg"), exist_ok=True)
        otp_file = path.join(folder, "gpkg", otp_file)
        df = gpd.read_file(zip_file)
        df.to_file(otp_file, driver="GPKG")

        # abre o arquivo zip com geopandas

    print("Arquivos descompactados com sucesso!")
else:
    print(f"O ano de {ano} não precisa descompactar")

Descompactando arquivos:   0%|          | 0/27 [00:00<?, ?arquivo/s]

ac_setores_censitarios.zip


Descompactando arquivos:   4%|▎         | 1/27 [00:00<00:09,  2.76arquivo/s]

al_setores_censitarios.zip


Descompactando arquivos:   7%|▋         | 2/27 [00:00<00:10,  2.35arquivo/s]

...

sp_setores_censitarios.zip


Descompactando arquivos:  96%|█████████▋| 26/27 [00:47<00:04,  4.43s/arquivo]

to_setores_censitarios.zip


Descompactando arquivos: 100%|██████████| 27/27 [00:47<00:00,  1.77s/arquivo]

Arquivos descompactados com sucesso!
